# Wiktionary - Stemmer und POS-Tagger

Autor: Christian Koss - ckoss@uni-bremen.de

In [1]:
# Verwendete externe Packages
import pandas as pd
print("Pandas-Version: \t"+pd.__version__)
import sqlite3
print("SQLite3-Version: \t"+sqlite3.sqlite_version)
import xml.etree.ElementTree as ET
print("xml.etree.ET-Version: \t"+ET.VERSION)
import re
print("RegEx-Version: \t\t"+re.__version__)
from collections import Counter

wiktionary_sql = 'data/wiktionary/wiktionary.db'

Pandas-Version: 	0.22.0
SQLite3-Version: 	3.11.0
xml.etree.ET-Version: 	1.3.0
RegEx-Version: 		2.2.1


In [2]:
# Lade/Parse Wiktionary
conn=sqlite3.connect(wiktionary_sql)
cur = conn.cursor()
cur.execute('SELECT title, text FROM wikipages')

bad_pages=['Hilfe:',
            'Kategorie:',
            'Modul:',
            'Reim:',
            'Verzeichnis:',
            'Vorlage:',
            'Flexion:',
            'Wiktionary:']

stemm={}
pos={}
synonym={}
related={}
gender={}
subs={}
token={}
extra={}
wortbildung={}
cat={}
lemma={}

#for title,text in cur:
#    if 'Verzeichnis:Deutsch/Wortbildungen/' in title:
#        print(title,text)
#        break;
test=[]
for title,text in cur:
    if text is not None and title.split(':')[0]+':' not in bad_pages and ('{{Sprache|Deutsch}}' in text or '|Deutsch}}' in text.split('===\n')[0] or '{{Alte Schreibweise' in text.split('===\n')[0]):
        
        for tag in re.findall('{{[^\{\|]{4,}}}',text):
            try:
                cat[tag]+=1
            except:
                cat[tag]=1
        temp=title.split('==')[0]
        token[temp]=None
        text=text.split('{{Sprache|Deutsch}}')[-1].split('== '+temp+' ({{')[0]
            
        #Stamm
        stemm[temp]=''
        try:
            matches=re.findall('{{[^{|]{0,}Grundformverweis[^}]+[|]([^|}]+)}',text)
            matches[0]
            matches=[t.split('#')[0] for t in matches]
            stemm[temp]+='|'.join(set([w.split('|')[0].split('}')[0].strip() for w in matches if '=' not in w]))
        except:
            pass
        try:
            matches=re.findall('{{[^{|]{0,}Schreibweise[^|}]{0,}[|]([^|]+)',text)
            matches[0]
            matches=[t.split('#')[0] for t in matches]
            stemm[temp]+='|'.join(set([w.split('|')[0].split('}')[0].strip() for w in matches if '=' not in w]))
        except:
            pass
            
        try:
            matches=re.findall('{{[^|]{0,}Lemmaverweis[^|}]{0,}[|]([^|}]+)', text)
            matches[0]
            matches=[t.split('#')[0] for t in matches]
            stemm[temp]+='|'.join(set([w.split('|')[0].split('}')[0].strip() for w in matches if '=' not in w]))               
        except:
            pass
        if stemm[temp]=='':
            stemm[temp]='None'
            

        #Gender
        for tag in ['{{Weibliche Wortformen}}','{{Männliche Wortformen}}','{{Sächliche Wortformen}}']:
            templist=[]
            try:
                templist+=re.findall('\[\[([^[]+)\]\]',re.search(tag+'[^{]+', text).group(0))
            except:
                pass
            if len(templist)>0:
                gender[temp]=templist
                
        #POS   
        try:
            matches=re.findall('{{Wortart[|]([^|}]+)', text)
            matches[0]
            pos[temp]='|'.join([w.split('}')[0].split('<!')[0].split('(')[0].strip() for w in matches])              
        except:
            pass
        
        #Lemmas
        try:
            synonym[temp]=re.findall('\[\[([^[]+)\]\][,\\n]',re.search('{{Synonyme}}[^{]+', text).group(0))
        except:
            pass
        
        try:
            related[temp]=re.findall('\[\[([^[]+)\]\][,\\n]',re.search('{{Sinnverwandte Wörter}}[^{]+', text).group(0))
        except:
            pass
        try:
            subs[temp]=re.findall('\[\[([^[]+)\]\][,\\n]',re.search('{{Unterbegriffe}}[^{]+', text).group(0))
        except:
            pass
        
        try:
            try:
                a=synonym[temp]
            except:
                a=[]
            try:
                b=related[temp]
            except:
                b=[]
            try:
                c=subs[temp]
            except:
                c=[]
            try:
                d=gender[temp]
            except:
                d=[]
                
            for word in a+b+c+d:
                try:
                    lemma[temp]+=[word]
                except:
                    lemma[temp]=[word]
                    token[temp]=None
        except:
            pass
            

        #extra
        try:
            wortbildung[temp]=[(a+b,c) for a,b,c in re.findall(""":''\[\[([A-Za-zäöüßÄÖÜ]+)\]\]:''|(:)|\[\[([^[]+)\]\]""",re.sub("""\]\]e:''""","""]]:''""",re.search('{{Wortbildungen}}([^{]+)', text).group(1)).replace('\n',''))]
        except:
            pass
        
        #if 'Turniere'==title:
        #    break;
conn.close()

examples={}
for k,v in pos.items():
    try:
        if len(examples[v])<3:
            examples[v]+=[k]
    except:
        examples[v]=[k]
        
len(token)

503242

In [3]:
#Erweitere den Datensatz mit Wörtern von Wiktionary ohne eigenständige Seite, leite den POS, wenn nicht erkennbar ab
unknown={}
count=0
for tok, wordlist in wortbildung.items():
    for tag, word in wordlist:
        try:
            pos[word]
        except:
            tag=tag.split('/')[0]
            word=word.split('/')[0]
            if tag==':':
                count+=1
                temp=count
            if tag!=':' and tag!='':
                temp=tag
                
            temp0=word.split(tok)    
            if len(temp0)==2 and len(temp0[0])>0 and len(temp0[1])==0 and pos[tok]=='Verb':
                pos[word]=pos[tok]
                token[word]=None
    
            else:
                if tag=='' and type(temp) is int:
                    if word[0].isupper():
                        try:
                            if pos[word]=='' or pos[word]==None:
                                pos[word]
                                token[word]=None
                        except:
                            pos[word]='Substantiv'
                            token[word]=None
                    else:
                        try:
                            unknown[temp]+=[(tok, temp,word)]
                        except:
                            unknown[temp]=[(tok, temp,word)]

                if tag=='' and type(temp) is not int:
                    try:
                        if pos[word]=='' or pos[word]==None:
                            pos[word]=temp
                            token[word]=None
                    except:
                        pos[word]=temp
                        token[word]=None
    wortbildung[tok]=[word for tag,word in wordlist if word!='']

In [4]:
#example
#stemm['grossporigen'],stemm['großporigen']

In [5]:
#unterster stamm
for tok in token:
    stemmx='None'   
    try:
        stemmx=stemm[tok]
    except:
        Next=False
    
    check=['None','']
    while stemmx not in check:
        try:
            if stemm[stemmx]!='None':
                stemmx=stemm[stemmx]
            check.append(stemmx)
        except:
            check.append(stemmx)
    stemm[tok]=stemmx

In [6]:
#example
#stemm['grossporigen'],stemm['großporigen'],stemm['überständen']

In [7]:
# Erstelle DataFrame
data=[]
for tok in token:
        try:
            pos0=pos[tok]
        except:
            pos0='None'
        
        try:
            stemm0=stemm[tok]
        except:
            stemm0='None'
            
        try:
            lemma0=lemma[tok]
        except:
            lemma0=['None']
            
        if lemma0=='':
            lemma0=['None']
            
        line=[tok.split('|')[0],pos0,stemm0,'|'.join(lemma0)]
        data.append(line)
            
df=pd.DataFrame(data, columns=['token','pos','stemm','lemma'])
df[:5]

,token,pos,stemm,lemma
0,unbedarfter,Deklinierte Form|Komparativ,unbedarft,None
1,falzet,Konjugierte Form,falzen,None
2,Kotaus,Deklinierte Form,Kotau,None
3,Visiers,Deklinierte Form,Visier,None
4,Ichthyopterygium,Substantiv,None,Protopterygium|Mesopterygium|Metapterygium


In [8]:
# Füge POS vom Stemm hinzu
df['stemm_pos']='None'
for line in df[(df['stemm'] != 'None')].iterrows():
    stemmpos='None'
    try:
        stemmx=line[1]['stemm']
    except:
        pass
    
    try:
        stemmpos=pos[stemmx]
    except:
        pass
    df.at[line[0], 'stemm_pos'] = stemmpos

In [9]:
# Falls kein POS und kein Stemm, wähle Lemma und füge es mit POS hinzu
df['lemma_stemm']='None'
df['lemma_pos']='None'
for line in df[(df['pos'] == 'None') & (df['stemm'] == 'None')].iterrows():
    
    lemma='None'
    lemmapos='None'   
    lemmas=False

    try:
        lemmas=line[1]['lemma'].split('|')
    except:
        pass

    for lemma in lemmas:
        try:
            lemmapos=pos[lemma]
            lemma=lemma[lemma]
            break;
        except:
            pass

    #if line[1]['token']=='Hauskeller':
    #    break;
    df.at[line[0], 'lemma_stemm'] = lemma
    df.at[line[0], 'lemma_pos'] = lemmapos

In [10]:
#Lösche nicht identifizierte token
df = df.drop(df[(df['pos'] == 'None') & (df['stemm'] == 'None') & (df['lemma'] == 'None')].index)

In [11]:
df['quick_stemm']='None'
df['quick_pos']='None'
#Erstelle Schnellzugriff für Stemm und POS

for line in df.iterrows():
    if line[1]['stemm_pos']!='None':
        temp = line[1]['stemm_pos']
    elif line[1]['pos']!='None':
        temp = line[1]['pos']
    elif line[1]['lemma_pos']!='None':
        temp = line[1]['lemma_pos']
    else:
        temp = 'None'
    
    temp=temp.lower()
    if 'negation' in temp:
        df.at[line[0], 'quick_pos'] = 'Negation'
    elif 'artikel' in temp and not 'partikel' in temp:
        df.at[line[0], 'quick_pos'] = 'Artikel'
    elif 'konjunktion' in temp:
        df.at[line[0], 'quick_pos'] = 'Konjunktion'
    elif 'buchstabe' in temp:
        df.at[line[0], 'quick_pos'] = 'Buchstabe'
    elif 'abkürzung' in temp:
        df.at[line[0], 'quick_pos'] = 'Abkürzung'
    elif 'substantiv' in temp:
        df.at[line[0], 'quick_pos'] = 'Substantiv'
    elif 'präposition' in temp:
        df.at[line[0], 'quick_pos'] = 'Präposition'
    elif 'superlativ' in temp:
        df.at[line[0], 'quick_pos'] = 'Superlativ'
    elif 'komparativ' in temp:
        df.at[line[0], 'quick_pos'] = 'Komparativ'
    elif 'adjektiv' in temp:
        df.at[line[0], 'quick_pos'] = 'Adjektiv'
    elif 'verb' in temp or 'partizip' in temp:
        df.at[line[0], 'quick_pos'] = 'Verb'
    else:
        df.at[line[0], 'quick_pos'] = temp
        
        
    if line[1]['lemma_stemm']!='None':
        df.at[line[0], 'quick_stemm'] = line[1]['lemma_stemm']
    elif line[1]['stemm']!='None':
        df.at[line[0], 'quick_stemm'] = line[1]['stemm']
    else:
        df.at[line[0], 'quick_stemm']  = line[1]['token']

In [12]:
Counter(list(df['quick_pos'])).most_common(30)

[('Substantiv', 286598),
 ('Verb', 158261),
 ('Adjektiv', 108176),
 ('Abkürzung', 4361),
 ('deklinierte form', 2721),
 ('redewendung', 2167),
 ('none', 1577),
 ('konjugierte form', 1347),
 ('deklinierte form|konjugierte form', 1139),
 ('dekliniertes gerundivum', 865),
 ('erweiterter infinitiv', 687),
 ('deklinierte form|deklinierte form', 546),
 ('gebundenes lexem', 478),
 ('Präposition', 473),
 ('konjugierte form|konjugierte form', 326),
 ('sprichwort', 156),
 ('suffix', 156),
 ('numerale', 138),
 ('interjektion', 135),
 ('präfix', 128),
 ('indefinitpronomen', 98),
 ('Komparativ', 90),
 ('ortsnamengrundwort', 86),
 ('Buchstabe', 69),
 ('Konjunktion', 68),
 ('konjugierte form|deklinierte form', 59),
 ('Superlativ', 54),
 ('straßenname|eigenname', 48),
 ('Artikel', 44),
 ('kontraktion', 39)]

In [13]:
df = df.drop(df[df['quick_pos'] == 'None'].index)

In [14]:
db='lexikon' 
conn = sqlite3.connect(wiktionary_sql)
cur = conn.cursor()                                 

wildcards = ','.join(['?'] * len(df.columns))              

cur.execute("drop table if exists %s" % db)

col_str = '"' + '","'.join(df.columns) + '"'
cur.execute("create table %s (%s)" % (db, col_str))

cur.executemany("insert into %s values(%s)" % (db, wildcards), [tuple(x) for x in df.values])

conn.commit()
conn.close()